In [1]:
#Importing libraries for webscraping & dataframe
import numpy as np 
import pandas as pd 
import codecs
from bs4 import BeautifulSoup
# Use wget to pull down the wikipedia page about Toronto Neighborhoods
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("Done")


--2021-01-08 21:38:49--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54733 (53K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M.2’

List_of_postal_code 100%[===================>]  53.45K  --.-KB/s    in 0.002s  

2021-01-08 21:38:50 (22.6 MB/s) - ‘List_of_postal_codes_of_Canada:_M.2’ saved [54733/54733]

Done


* Now read the resulting file into a string.
* webscraping using Beautiful Soup.
* convert the table portion into dataframe
* name the columns :: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.



In [3]:
f = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = f.read()
soup = BeautifulSoup(html, 'html.parser')

df_canada = (pd.read_html(str(soup.table))[0])

#drop the first row, which is the headings of the table
df_canada.drop(axis=0, index=0, inplace=True)

#then assign the proper column names
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df_canada.columns = column_names

#get rid of unassigned Boroughs
df_canada = df_canada[df_canada['Borough'] != "Not assigned"].reset_index(drop=True) 
df_canada.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


* If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [4]:
not_assigned = df_canada[df_canada['Neighborhood']=="Not assigned"]
for i in range(len(not_assigned)):
    info = not_assigned.iloc[index]
    info['Neighborhood']=info['Borough']
    df_canada.iloc[not_assigned.index[0]]['Neighborhood'] = info['Borough']

df_canada.head(8)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills


* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [5]:
df_canada = df_canada.groupby(['Borough', 'PostalCode'], as_index=False).agg(','.join)
df_canada.head(10)

,Borough,PostalCode,Neighborhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"
5,Central Toronto,M4V,"Summerhill West, Rathnelly, South Hill, Forest..."
6,Central Toronto,M5N,Roselawn
7,Central Toronto,M5P,"Forest Hill North & West, Forest Hill Road Park"
8,Central Toronto,M5R,"The Annex, North Midtown, Yorkville"
9,Downtown Toronto,M4W,Rosedale


* .shape method to print the number of rows of your dataframe.

In [6]:
#Shape of Data
df_canada.shape

(103, 3)